In [4]:
from pymongo import MongoClient

In [5]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [6]:
dbAer, collAer = connectCollection('getlocation','aeropuertos')

In [7]:
dbMO, collMO = connectCollection('getlocation','myOfficesFarOlds')

In [8]:
aer = list(collAer.find())
aer[0]

{'_id': ObjectId('5dd5a578e49f64fa8f937f98'),
 'id': 6523,
 'ident': '00A',
 'type': 'heliport',
 'name': 'Total Rf Heliport',
 'latitude_deg': 40.07080078125,
 'longitude_deg': -74.93360137939453,
 'elevation_ft': 11,
 'continent': 'NA',
 'iso_country': 'US',
 'iso_region': 'US-PA',
 'municipality': 'Bensalem',
 'scheduled_service': 'no',
 'gps_code': '00A',
 'iata_code': '',
 'local_code': '00A',
 'home_link': '',
 'wikipedia_link': '',
 'keywords': '',
 'location': {'type': 'Point',
  'coordinates': [-74.93360137939453, 40.07080078125]}}

In [9]:
def getLejosAero (longitud,latitud):
    myOfAer = collAer.find(
        {"location":
         {"$near":
          {"$geometry":
           {"type":"Point",
            "coordinates": [longitud,latitud]
           },
           "$maxDistance":30000,
          }
         }
        }
    ).limit(1)
    return myOfAer

In [10]:
myOff=list(collMO.find())
myOff[0]['offices']['longitude']

10.023246

In [11]:
list(getLejosAero(float(myOff[1]['offices']['longitude']),float(myOff[1]['offices']['latitude'])))

[{'_id': ObjectId('5dd5a57ae49f64fa8f94447d'),
  'id': 328919,
  'ident': 'US-0938',
  'type': 'heliport',
  'name': 'Advocate Sherman Hospital Heliport',
  'latitude_deg': 42.072178,
  'longitude_deg': -88.329757,
  'elevation_ft': 905,
  'continent': 'NA',
  'iso_country': 'US',
  'iso_region': 'US-IL',
  'municipality': 'Elgin',
  'scheduled_service': 'no',
  'gps_code': '25LL',
  'iata_code': '',
  'local_code': '25LL',
  'home_link': '',
  'wikipedia_link': '',
  'keywords': 'Sherman Hospital Randall Road',
  'location': {'type': 'Point', 'coordinates': [-88.329757, 42.072178]}}]

In [12]:
for i in range(len(myOff)):
    value = {'$set':{'aeropuertoMasCercano':list(getLejosAero(float(myOff[i]['offices']['longitude']),float(myOff[i]['offices']['latitude'])))}}
    collMO.update_one(myOff[i],value)

In [20]:
dbDef, collDef = connectCollection('Opciones','misOpciones')

In [27]:
x=list(collDef.find())
len(x)
x[0]['location']['coordinates'][::-1]

[54.703145, -1.574948]

In [15]:
import folium

In [41]:
map_city=folium.Map(zoom_start=15)
for e in range(len(x)):
    folium.Marker(x[e]['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)
    folium.Marker(x[e]['aeropuertoMasCercano'][0]['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='blue'), 
                   ).add_to(map_city)
    folium.Marker(x[e]['StbckMasCercano'][0]['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='purple'), 
                   ).add_to(map_city)
    
map_city